In [26]:
import json
import sys
import os
sys.path.append(os.environ["BUILD_WORKSPACE_DIRECTORY"])


In [27]:
from core.infra.cloud.gcs_cv2_utils import upload_cv2_image_to_gcs
from core.infra.cloud.gcs_utils import dump_to_gcs, get_storage_client
output_path = "gs://voxel-users/reza/ergo_syntheticdata_downsample"
storage_client = get_storage_client()

In [28]:
import numpy as np
import torch
import cv2
from PIL import Image
import os
import matplotlib.pyplot as plt
from torchvision import transforms
transform = transforms.ColorJitter(brightness=(0.3,1.2))
def get_downsample(data, dict_sample, max_id, root_dir):
    annotations = data['annotations'].copy()
    annotations_new = []
    annotations_new.extend(annotations)
    save_subfolder = root_dir.split('/')[-1]
    for img_name, annos in dict_sample.items():
        sample_image = Image.open(os.path.join(root_dir, img_name))
        img_array = np.asarray(sample_image)[...,:3]
        sample_image = Image.fromarray(img_array.astype('uint8'), 'RGB')
        upload_cv2_image_to_gcs(
                     cv2.cvtColor(np.asarray(sample_image), cv2.COLOR_BGR2RGB),
                     f"{output_path}/{save_subfolder}/{img_name}",
                    storage_client=storage_client,
                )
        for i in [0.75, 0.5, 0.25]:
            max_id = max_id + 1
            new_width = int(sample_image.width * i)
            new_height = int(sample_image.height * i)
            Image1copy = sample_image.resize((new_width, new_height), Image.ANTIALIAS)
            image_name1 = f"scale_{i}_{img_name}"
            occlude_key1 = {'id': max_id, 'file_name': image_name1, 'width': Image1copy.width, 'height': Image1copy.height}
            data['images'].append(occlude_key1)
            for anno in annos:
                if anno['category_id'] == 0:
                    anno_new = anno.copy()
                    anno_new['bbox'] = [int(x * i) for x in anno_new['bbox']]
                    anno_new['image_id'] = occlude_key1['id']
                    annotations_new.append(anno_new)
            upload_cv2_image_to_gcs(
                     cv2.cvtColor(np.asarray(Image1copy), cv2.COLOR_BGR2RGB),
                     f"{output_path}/{save_subfolder}/{image_name1}",
                    storage_client=storage_client,
                )
    data['annotations'] = annotations_new
    gcs_label_path = f"{output_path}/{save_subfolder}/annotations.json"
    dump_to_gcs(
            gcs_label_path,
            json.dumps(data),
            content_type="application/json",
    )
    

In [29]:
root_dir = '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/'
data_dirs = [x[0] for x in os.walk(root_dir)][1:]
print(data_dirs)

['/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-bloomingdale-0002', '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-bloomingdale-0005', '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-pooler-0004', '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_uscold-laredo-dock01', '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-pooler-0008', '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-pooler-0007', '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-bloomingdale-0007', '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-bloomingdale-0006', '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-bloomingdale-0004', '/home/reza_voxelsafety_com/experiments/ergonomic/er

In [30]:
for i in range(len(data_dirs)):
    print(data_dirs[i])
    data = json.load(open(f"{data_dirs[i]}/annotations.json"))
    dict_sample = {}
    for name in data['images']:
        sample = [anno for anno in data['annotations'] if anno['image_id'] == name['id']]
        dict_sample[name['file_name']] = sample
    ids = [imgdata['id'] for imgdata in data['images']]
    max_id = max(ids)
    get_downsample(data, dict_sample, max_id, data_dirs[i])
    

/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-bloomingdale-0002
/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-bloomingdale-0005
/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-pooler-0004
/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_uscold-laredo-dock01
/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-pooler-0008
/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-pooler-0007
/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-bloomingdale-0007
/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-bloomingdale-0006
/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-bloomingdale-0004
/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-s

In [8]:
A = [1, 2, 3, 4]

In [9]:
A = [x / 10 for x in A]
print(A)

[0.1, 0.2, 0.3, 0.4]
